In [22]:
%matplotlib inline

import numpy as np
import pyvista as pv
from scipy.spatial import KDTree
from binary_to_csv import binary_to_csv
import os
import pandas as pd

In [23]:
# DATA_DIR = '../data'
# BIN_FILE_NAME = "data_20241015_183835.bin"

# BIN_FILE_PATH = os.path.join(DATA_DIR, BIN_FILE_NAME)
# CSV_FILE_PATH = os.path.join(DATA_DIR, BIN_FILE_NAME.replace(".bin", ".csv"))


# binary_to_csv(BIN_FILE_PATH, CSV_FILE_PATH)
# print(CSV_FILE_PATH)

df = pd.read_csv("../data/results/1-data_20241115_120617_post.csv", header=None)

# DATA_DIR = '../data/results'
# csvs = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
# csvs.sort()


# df = pd.read_csv(os.path.join(DATA_DIR, csvs[7]), header=None)

In [24]:
ASSETS_DIR = '../assets'

grid_points = np.load(os.path.join(ASSETS_DIR, "boob_grid_12_21.npy"))

# Remove points that are outside of the boob mesh.
# Because those points have been skipped during the points mapping, their value is (0, 0, 0)
origin = np.array([0, 0, 0])
points_to_skip = np.where((grid_points == origin).all(axis=1))[0]
points_of_interest = np.delete(grid_points, points_to_skip, axis=0)

# Drop the columns that represent points out of the breast model.
df_poi = df.drop(columns=points_to_skip)

df_poi.head()

# df_poi = df_poi.applymap(lambda x: x if x > 100 else 0)

,2,3,4,5,14,15,16,17,18,19,...,238,242,243,244,245,246,247,248,249,250
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,-8,...,-10,-7,7,-6,-11,0,-15,-3,-11,3
2,-4,3,2,3,-7,-7,-4,9,2,-5,...,-10,-7,6,7,7,-3,1,13,7,-1
3,2,7,10,3,6,10,-5,2,6,3,...,8,7,8,2,4,3,3,2,2,3
4,1,14,2,5,-8,-10,-1,3,3,1,...,-13,-14,6,-10,5,-12,1,3,-3,-11


In [25]:
# Create the point cloud and initialize the scalars
point_cloud = pv.PolyData(points_of_interest)

p1 = pv.Plotter(border=False)
p1.add_title("Breast points", font_size=12)
p1.add_mesh(point_cloud, copy_mesh=True ,scalars=np.zeros(points_of_interest.shape[0]), cmap='cool', point_size=15)

p1.remove_scalar_bar()

p1.camera_position = 'xy'
p1.camera.elevation += 100

# p1.show()


In [26]:
K = 5

# Load the boob 3D model from file
boob_mesh = pv.read( '../assets/boob.obj')
boob_mesh.translate(np.array([0, -0.035, 0]), inplace=True)

# Create a kd-tree for quick nearest-neighbor lookup.
kdtree = KDTree(points_of_interest)

# Find the K nearest point_cloud points for each points in the boob mesh and calculate their respective distances
boob_mesh['nearest_points'] = kdtree.query(boob_mesh.points, k=K)[1]
boob_mesh['nearest_points_dist'] = kdtree.query(boob_mesh.points, k=K)[0]

# Step 1: Calculate the reciprocal of the distances
reciprocal_distances = 1 / boob_mesh['nearest_points_dist']

# Step 2: Square the reciprocal distances
squared_reciprocal_distances = reciprocal_distances ** 1.2


# Step 3: Normalize the weights
normalized_weights = squared_reciprocal_distances / \
    np.sum(squared_reciprocal_distances, axis=1, keepdims=True)

boob_mesh['pressure'] = np.zeros(boob_mesh.points.shape[0])
boob_mesh['max_pressure'] = boob_mesh['pressure']


p4 = pv.Plotter(border=False)
p4.add_title("Breast model", font_size=12)
p4.add_mesh(boob_mesh, copy_mesh=True , scalars=np.zeros(boob_mesh.points.shape[0]), cmap='cool', point_size=15)

p4.remove_scalar_bar()

p4.camera_position = 'xy'
p4.camera.elevation += 100

# p4.show()



In [27]:
calibration_data = df_poi.iloc[:20].median()

In [28]:
def interpolate_grid_on_model(grid_scalar):
    return np.sum(grid_scalar[boob_mesh['nearest_points']] * normalized_weights, axis=1)

In [29]:
THRESHOLD = 100

p2 = pv.Plotter( border=False, shape=(1, 2))

points_touched = (df_poi > calibration_data + THRESHOLD).any().values
point_cloud['pressed'] = points_touched

p2.subplot(0, 0)
p2.add_title("Points pressed", font_size=8)
p2.add_mesh(point_cloud, copy_mesh=True, scalars='pressed', cmap='cool', point_size=10)


s = np.sum(points_touched[boob_mesh['nearest_points']] * normalized_weights, axis=1)


p2.subplot(0, 1)
p2.add_title("Points pressed", font_size=8)
p2.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(points_touched), cmap='cool', point_size=10)

p2.link_views()

p2.camera_position = 'xy'
p2.camera.elevation += 100



p2.show()

Widget(value='<iframe src="http://localhost:62305/index.html?ui=P_0x32db13200_12&reconnect=auto" class="pyvist…

In [30]:

relative_pressure = df_poi - calibration_data


p3 = pv.Plotter(border=False, shape=(2, 2))

p3.subplot(0, 0)
p3.add_title("Mean Pressure", font_size=8)
p3.add_mesh(point_cloud, copy_mesh=True, scalars=relative_pressure.mean(), scalar_bar_args={
             'title': 'mean', "title_font_size": -1}, cmap='cool', point_size=10)

p3.subplot(1, 0)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(relative_pressure.mean().values), scalar_bar_args={
             'title': 'mean', "title_font_size": -1}, cmap='cool', point_size=10)

p3.subplot(0, 1)
p3.add_title("Max Pressure", font_size=8)
p3.add_mesh(point_cloud, copy_mesh=True, scalars=relative_pressure.max(), scalar_bar_args={
             'title': 'max', "title_font_size": -1}, cmap='cool', point_size=10)

p3.subplot(1, 1)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(relative_pressure.max().values), scalar_bar_args={
             'title': 'max', "title_font_size": -1}, cmap='cool', point_size=10)

p3.link_views()


p3.camera_position = 'xy'
p3.camera.elevation += 100

p3.show(jupyter_backend='trame')



Widget(value='<iframe src="http://localhost:62305/index.html?ui=P_0x30f1720f0_13&reconnect=auto" class="pyvist…

: 